# Generate Data
***

#### Overview
This notebook creates 2 datasets. One dataset contains all stock prices with RSI values and signals. The second dataset holds information for trades only.

#### Output

Dataset for all prices in AAPL_RSI_Signals.csv:

| DateTime | Adj Close | Volume | RSI | Signal |
| ----- | ----- | ----- | ----- | ----- |

Dataset for trades in trades.csv:

| EntryPrice | ExitPrice | EntryDateTime | ExitDateTime | PnL ($) | PnL (\%) | HoldingPeriod |
| ----- | ----- | ----- | ----- | ----- | ----- | ----- |

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import yfinance as yf
import talib
import os

### 1. Download data from yFinance

In [2]:
# Define functions

def getData(ticker, period="max", interval="1d"):
    data = yf.download(tickers=ticker, period=period, interval=interval)
    return data

def saveData(ticker, data):
    os.makedirs("Data", exist_ok=True)
    saveLocation = "Data/" + ticker + '.csv'
    data.to_csv(saveLocation, index_label='DateTime')

### 2. Functions to process and transform data

### A) Compute RSI values

In [3]:
def getRSI(data):
    # Get list of prices
    adjClose = data['Adj Close'].values
    
    # Compute list of RSI values
    rsiValues = talib.RSI(adjClose, timeperiod=14)
    
    return rsiValues

### B) Add RSI to stock price dataset

In [4]:
def addRSI(rsiValues, data):
    # Add RSI column with values to stock price dataset
    data['RSI'] = rsiValues
    
    # Remove first 14 rows that don't have RSI values
    dataRSI = data.dropna(subset=['RSI']).copy()
    
    return dataRSI

### C) Get trade signals
- if RSI < 30: Buy
- if RSI > 70: Sell
- else: hold or do nothing

In [5]:
def getTradeSignals(data):
    '''
    Add Signal column to dataset 
    based on RSI strategy
    '''
    numDates = data.shape[0]
    data["Signal"] = ""
    holdingPeriod = False
    
    for i in range(numDates):
        
        RSI = data['RSI'].iloc[i]
        price = data['Adj Close'].iloc[i]
        
        if RSI < 30 and holdingPeriod==False:
            data["Signal"].iloc[i] = "Buy"
            holdingPeriod = True
            
        elif RSI > 70 and holdingPeriod==True:
            data["Signal"].iloc[i] = "Sell"
            holdingPeriod = False
    
    return data

### D) Create new dataset

In [6]:
def saveStockData(ticker, data):
    '''
    Save subset of data with only Adjusted close prices
    '''
    data = data[['DateTime', 'Adj Close', 'Volume', 'RSI', 'Signal']]
    #data = data.reset_index() #make 'DateTime' column from index
    
    # Save as csv
    data.to_csv(ticker+'_RSI_Signals.csv')
    
    return data

In [7]:
def getTradesData(data):
    
    # Filter for rows with buy and sell signals only
    temp = data[data['Signal'] != ""]
    
    # Create new dataframe
    cols = ['EntryDateTime', 'ExitDateTime', 'EntryPrice', 'ExitPrice', 
               'DollarPnL', 'PercentPnL', 'HoldingDuration']
    tradesData = pd.DataFrame(columns=cols)
    
    # Generate new data
    for i, trade in temp.groupby(np.arange(len(temp)) // 2):
        # Buy
        EntryDateTime = trade['DateTime'][trade['Signal']=="Buy"].values[0]
        EntryPrice = trade['Adj Close'][trade['Signal']=="Buy"].values[0]
        # Sell
        ExitDateTime = trade['DateTime'][trade['Signal']=="Sell"].values[0]
        ExitPrice = trade['Adj Close'][trade['Signal']=="Sell"].values[0]
        #PnL
        DollarPnL = ExitPrice - EntryPrice
        PercentPnL = DollarPnL/EntryPrice
        # HoldingDuration in days
        timedelta = ExitDateTime - EntryDateTime
        HoldingDuration = timedelta.astype('timedelta64[D]').astype(int)
        
        # Add row of data to new dataframe
        rowList = [[EntryDateTime, ExitDateTime, EntryPrice, ExitPrice, DollarPnL, PercentPnL, HoldingDuration]]
        row = pd.DataFrame(rowList, columns=cols)
        tradesData = pd.concat([row, tradesData], ignore_index=True)
        
    # Reverse order so that it is goes from oldest to newest dates
    tradesData = tradesData[::-1].reset_index(drop=True)
    
    # Save as csv
    tradesData.to_csv("trades.csv")
    
    return tradesData

## Run program

### Download data

In [8]:
# Download data from API
ticker = "AAPL"
data = getData(ticker) #get data from Yahoo Finance
saveData(ticker, data) #save data to "Data" folder

# View data
print(data.shape)
data.head(3)

[*********************100%***********************]  1 of 1 completed
(10333, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100453,469033600
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095213,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088224,105728000


### Generate trades

In [9]:
# Load data from file
aapl = "Data/AAPL.csv"
data = pd.read_csv(aapl, parse_dates=['DateTime'])
data.head(3)

,DateTime,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100453,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095213,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088224,105728000


In [10]:
# Compute RSI values and add as new column
RSIValues = getRSI(data)
RSIData = addRSI(RSIValues, data)

# Find trading signals based on RSI strategy
SignalsData = getTradeSignals(RSIData)

/home/saloni/anaconda3/envs/4122Project/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [11]:
SignalsData.head()

,DateTime,Open,High,Low,Close,Adj Close,Volume,RSI,Signal
14,1981-01-05,0.151228,0.151228,0.150670,0.150670,0.117924,35728000,64.493251,
15,1981-01-06,0.144531,0.144531,0.143973,0.143973,0.112683,45158400,58.970446,
16,1981-01-07,0.138393,0.138393,0.137835,0.137835,0.107879,55686400,54.374540,
17,1981-01-08,0.135603,0.135603,0.135045,0.135045,0.105695,39827200,52.376301,
18,1981-01-09,0.142299,0.142857,0.142299,0.142299,0.111372,21504000,56.819436,


### Compute and save PnL for trades

In [12]:
# Get and save subset of data with only Adjusted Close prices
SignalsData = saveStockData(ticker, SignalsData)

# Get and save trades data 
tradesData = getTradesData(SignalsData)

In [13]:
#Results
tradesData.head()

,EntryDateTime,ExitDateTime,EntryPrice,ExitPrice,DollarPnL,PercentPnL,HoldingDuration
0,1981-03-11,1981-05-22,0.075558,0.109625,0.034067,0.450869,72
1,1981-07-01,1981-12-18,0.089971,0.079927,-0.010045,-0.111643,170
2,1982-03-05,1982-08-23,0.058089,0.053721,-0.004368,-0.075196,171
3,1983-07-19,1983-12-16,0.152865,0.086477,-0.066387,-0.434288,150
4,1984-10-12,1985-10-16,0.079490,0.062893,-0.016597,-0.208796,369


In [14]:
tradesData.tail()

,EntryDateTime,ExitDateTime,EntryPrice,ExitPrice,DollarPnL,PercentPnL,HoldingDuration
27,2018-02-02,2018-05-09,38.376293,44.981293,6.605000,0.172111,96
28,2018-11-20,2019-03-13,42.952045,44.289131,1.337086,0.031130,113
29,2019-05-31,2019-10-21,42.835052,59.070087,16.235035,0.379013,143
30,2020-02-27,2020-06-05,67.539055,82.077385,14.538330,0.215258,99
31,2021-02-25,2021-06-30,120.428375,136.555817,16.127441,0.133917,125
